# Spectrogram

In [1]:
import musdb
mus_train = musdb.DB(download=True, subsets='train', split='train')
mus_valid = musdb.DB(download=True, subsets='train', split='valid')
mus_test = musdb.DB(download=True, subsets='test')

print('track numbers of mus_train: {}'.format(len(mus_train)))
print('track numbers of mus_valid: {}'.format(len(mus_valid)))
print('track numbers of mus_test: {}'.format(len(mus_test)))

track numbers of mus_train: 80
track numbers of mus_valid: 14
track numbers of mus_test: 50


In [135]:
import random
from torch.utils.data import Dataset, DataLoader

class RandomMusdbTrainDataset(Dataset):

    def __init__(self, musdb, sample_size):
        self.musdb = musdb
        self.sample_size = sample_size

    def __len__(self) -> int:
        return len(self.musdb)

    def __getitem__(self, i):
        mixture = self.musdb[i].audio.T # [2, T]
        length = mixture.shape[1]
        start_pos = random.randint(0, length - self.sample_size -1)
        print(i, start_pos)
        mixture[:, start_pos: start_pos + self.sample_size]
        
        targets = {source: self.musdb[i].sources[source].audio.T[:, start_pos: start_pos + self.sample_size]
                   for source 
                   in self.musdb[i].sources.keys()}
        
        return mixture[:, start_pos: start_pos + self.sample_size], targets
    
dataloader = DataLoader(RandomMusdbTrainDataset(mus_train, sample_size=2**18))

In [12]:
import torch
import torch.nn as nn
import pytorch_lightning


class SpecModel(pl.LightningModule):
    def __init__(self, nfft=1024, hop_length=512, depth=4, channels=8):
        super().__init__()
        self.nfft=nfft
        self.hop_length=hop_length

        self.encoder, self.decoder = [], []
        in_channels, out_channels= 2,4

        
        for index in range(depth):
            encode = [nn.Conv2d(in_channels, out_channels, (2,2), (2,2), dtype=torch.complex), nn.ReLU()]
            self.encoder.append(nn.Sequential(*encode))

            decode = [nn.ConvTranspose2d(out_channels, in_channels, (2,2), (2,2), dtype=torch.complex)]
            if index > 0:
                decode.append(nn.ReLU())

            self.decoder.insert(0, nn.Sequential(*decode))
            
            in_channels = out_channels
            out_channels = 2 * out_channels
            


In [173]:
from torch import nn

nn.Linear(4,6, dtype=torch.complex64)(spec)

<module 'torch.nn' from '/home/wschoi/exit/envs/tutorial-environment/lib/python3.9/site-packages/torch/nn/__init__.py'>

In [80]:
from IPython.display import Audio, display

In [167]:
dataloader = DataLoader(RandomMusdbTrainDataset(mus_train, sample_size= (2**9) * (2**8 -1)))

for mixture, target_dict in dataloader:
    display(Audio(mixture.cpu().detach().numpy()[0], rate=44100))
    break

0 151054


In [168]:
mixture.shape

torch.Size([1, 2, 130560])

In [175]:
spec = torch.stft(mixture[0], n_fft=1024, hop_length=512, return_complex=True)
spec.shape

torch.Size([2, 513, 256])

In [171]:
sepc = torch.view_as_complex(spec)
torch.istft(spec, n_fft=1024, hop_length=512, return_complex=False).shape

torch.Size([2, 130560])